# Tensorflow Implementation of VGG Face Detector

Objective: Convert the VGG face descriptor model http://www.robots.ox.ac.uk/~vgg/software/vgg_face/ to Tensorflow format.
<br>
Output: Final Tensorflow classifier model trained on the gender dataset

In [33]:
import scipy.io as sio
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import ZeroPadding2D, Convolution2D, MaxPooling2D, Dense, Dropout, Softmax, Flatten, Activation, BatchNormalization

## Read .mat file for Model Description

In [34]:
def load_mat():
    mat = sio.loadmat('data/vgg_face.mat', struct_as_record=False)
    
net = mat['net'][0][0]
mat_model = net.layers
mat_model_layers = mat_model[0]
num_mat_layers = mat_model_layers.shape[0]
for layer in mat_model_layers:
    print(layer[0][0].name)

['conv1_1']
['relu1_1']
['conv1_2']
['relu1_2']
['pool1']
['conv2_1']
['relu2_1']
['conv2_2']
['relu2_2']
['pool2']
['conv3_1']
['relu3_1']
['conv3_2']
['relu3_2']
['conv3_3']
['relu3_3']
['pool3']
['conv4_1']
['relu4_1']
['conv4_2']
['relu4_2']
['conv4_3']
['relu4_3']
['pool4']
['conv5_1']
['relu5_1']
['conv5_2']
['relu5_2']
['conv5_3']
['relu5_3']
['pool5']
['fc6']
['relu6']
['dropout6']
['fc7']
['relu7']
['dropout7']
['fc8']
['softmax']


## VGG Architecture Implementation
We can read the model described in the .mat file and build our tensorflow/keras model using clues. This way we can avoid reproducing it by hand. Additionally, this saves us the trouble of later applying the model weights from the .mat file to our tensorflow/keras model.

In [41]:
def vgg_tf():
    model = Sequential()

    for i in range(num_mat_layers):
        mat_model_layer = mat_model_layers[i][0][0].name[0]
        if mat_model_layer.find("conv") == 0 or mat_model_layer.find("fc") == 0:
            weights = mat_model_layers[i][0,0].weights
            weights_shape = weights[0][0].shape
            filter_x = weights_shape[0]; filter_y = weights_shape[1]
            number_of_filters = weights_shape[3]

            if mat_model_layer.find("conv") == 0:
                print("ZeroPadding2D((1,1))")
                if i == 0:
                    model.add(ZeroPadding2D((1,1),input_shape=(224,224, 3)))
                else:
                    model.add(ZeroPadding2D((1,1)))

            print("Convolution2D(",number_of_filters,", (",filter_x,", ",filter_y,"), name='",mat_model_layer,"')")
            model.add(Convolution2D(number_of_filters, (filter_x, filter_y), name= mat_model_layer))

        else:
            if mat_model_layer.find("relu") == 0:
                print("Activation('relu', name=",mat_model_layer)
                model.add(Activation('relu', name=mat_model_layer))
            elif mat_model_layer.find("dropout") == 0:
                print("Dropout(0.5, name=",mat_model_layer,")")
                model.add(Dropout(0.5, name=mat_model_layer))
            elif mat_model_layer.find("pool") == 0:
                print("MaxPooling2D((2,2), strides=(2,2), name=",mat_model_layer,")")
                model.add(MaxPooling2D((2,2), strides=(2,2), name=mat_model_layer))
            elif mat_model_layer.find("softmax") == 0:
                print("Activation('softmax', name=",mat_model_layer,")")
                model.add(Activation('softmax', name=mat_model_layer))
            else:
                print("-->",mat_model_layer)
    return model

Save model to .h5 file for use later.

In [42]:
model = vgg_tf()
model.save('data/vgg_face.h5')

ZeroPadding2D((1,1))
Convolution2D( 64 , ( 3 ,  3 ), name=' conv1_1 ')
Activation('relu', name= relu1_1
ZeroPadding2D((1,1))
Convolution2D( 64 , ( 3 ,  3 ), name=' conv1_2 ')
Activation('relu', name= relu1_2
MaxPooling2D((2,2), strides=(2,2), name= pool1 )
ZeroPadding2D((1,1))
Convolution2D( 128 , ( 3 ,  3 ), name=' conv2_1 ')
Activation('relu', name= relu2_1
ZeroPadding2D((1,1))
Convolution2D( 128 , ( 3 ,  3 ), name=' conv2_2 ')
Activation('relu', name= relu2_2
MaxPooling2D((2,2), strides=(2,2), name= pool2 )
ZeroPadding2D((1,1))
Convolution2D( 256 , ( 3 ,  3 ), name=' conv3_1 ')
Activation('relu', name= relu3_1
ZeroPadding2D((1,1))
Convolution2D( 256 , ( 3 ,  3 ), name=' conv3_2 ')
Activation('relu', name= relu3_2
ZeroPadding2D((1,1))
Convolution2D( 256 , ( 3 ,  3 ), name=' conv3_3 ')
Activation('relu', name= relu3_3
MaxPooling2D((2,2), strides=(2,2), name= pool3 )
ZeroPadding2D((1,1))
Convolution2D( 512 , ( 3 ,  3 ), name=' conv4_1 ')
Activation('relu', name= relu4_1
ZeroPadding2D((

## Construct Classifier 

In [39]:
def classier_model(input_dim):
    classifier = Sequential()
    classifier.add(Dense(100, input_dim=input_dim)) #input dim
    classifier.add(BatchNormalization())
    classifier.add(Activation('tanh'))
    classifier.add(Dropout(0.3))
    classifier.add(Dense(20))
    classifier.add(BatchNormalization())
    classifier.add(Activation('tanh'))
    classifier.add(Dropout(0.1))
    classifier.add(Dense(140))
    classifier.add(Activation('softmax'))
    classifier.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(), optimizer='adam', metrics=['accuracy'])
    return classifier

## Load Images

## Train Classifier

## Test Model

## Results